In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json
import h5py

srcDir = r"d:/OneDrive - The University of Western Ontario/Documents/PhD/Thesis/CodeRepositories/windCalc/src"
if srcDir not in sys.path:
    sys.path.append(srcDir)

import windIO
import wind
import windWT as wt
import windCAD
# import windOF as foam
import windCodes
import structure as st

import winsound

from matplotlib.ticker import FuncFormatter

In [24]:
B = 0.2744
D = 0.183
lScl=1/50.0

Hr = 0.08  # ridge height
roofSlope_rad = np.arctan(0.25/12)  # roof slope
roofSlope = np.degrees(roofSlope_rad)  # roof slope
He = Hr - D/2.0*np.sin(roofSlope_rad)  # eaves height
Havg = np.mean([Hr, He])
H = Havg  # average roof height
g = 0.016 # exploded plot gaps

# print("Hr \t= {:.5g} m,\nHe \t= {:.5g} m,\nHavg \t= {:.5g} m".format(Hr, He, Havg))
# print(f"Hr = {Hr:.5g} m,\nHe = {He:.5g} m,\nHavg = {Havg:.5g} m,\nH = {H:.5g} m,\ng = {g:.5g} m,\nB = {B:.5g} m,\nD = {D:.5g} m")

# print(f"B \t= {B:.5g} m,\nD \t= {D:.5g} m,\nrf.sl.\t= {roofSlope:.5g} deg.\nH \t= {H:.5g} m,\nHr \t= {Hr:.5g} m,\nHe \t= {He:.5g} m,\nHavg \t= {Havg:.5g} m\ng \t= {g:.5g} m,\nlScl \t= 1:{1/lScl:.5g},")
print(f"B \t= {B/lScl:.5g} m,\nD \t= {D/lScl:.5g} m,\nrf.sl.\t= {roofSlope:.5g} deg.\nH \t= {H/lScl:.5g} m,\nHr \t= {Hr/lScl:.5g} m,\nHe \t= {He/lScl:.5g} m,\nHavg \t= {Havg/lScl:.5g} m\ng \t= {g/lScl:.5g} m,")

areaFactor_toFSm2 = (1/lScl**2)   # convert mm2 to m2 and normalize by the square of the length scale

fluidDensity = 1.225  # only for wind tunnel
fluidKinematicViscosity = 1.48e-5

exportFolder = r"D:/tempData_depot/simData_CandC/MWFRS_info/"


B 	= 13.72 m,
D 	= 9.15 m,
rf.sl.	= 1.1935 deg.
H 	= 3.9524 m,
Hr 	= 4 m,
He 	= 3.9047 m,
Havg 	= 3.9524 m
g 	= 0.8 m,


In [25]:
Nf = 5  # number of frames
Npw = 5 # number of panels for walls (per eave height)
N_elem_perFrame = 4
face_IDs = [6,1,2,4] #[4,2,1,6]

rs = roofSlope_rad
roofInclineLength = D/2/np.cos(rs)
plw = He/Npw # panel length for walls

N_panelPerRoofIncline = int(np.ceil(roofInclineLength/plw))
plr = roofInclineLength/N_panelPerRoofIncline # panel length for roof

print(f"roofSlope_rad = {rs:.5g} rad,\nroofSlope = {roofSlope:.5g} deg.")
print(f"roofInclineLength = {roofInclineLength:.5g} m")
print(f"plw = {plw:.5g} m,\nplr = {plr:.5g} m,\nN_panelPerRoofIncline = {N_panelPerRoofIncline}")

roofSlope_rad = 0.02083 rad,
roofSlope = 1.1935 deg.
roofInclineLength = 0.09152 m
plw = 0.015619 m,
plr = 0.015253 m,
N_panelPerRoofIncline = 6


In [26]:
'''
------------------------------------------------------------------------------------------
===================================== NODES ==============================================
------------------------------------------------------------------------------------------
'''
# generate coordinates for all nodes in the form of a dataframe
# For a single frame:
# the frame is a portal frame ABCDE with coordinates of [x, -D/2, 0], [x, -D/2, He], [x, 0, Hr], [x, D/2, He], [x, D/2, 0], where x is the frame coordinate, D is the building width, He is the eaves height, and Hr is the ridge height 
# the frame is divided into Npw panels for walls and Npr panels for roofs

# total number of nodes per frame = 2*(Npw + Npr) + 1
# node ID is 0-based and continuous across all frames
# node name is only given to the main nodes at the start, end, or bends of the frame. A portal frame has 5 main nodes named A,B,C,D, & E, respectively, and the rest are intermediate nodes with no name
# frame ID is the frame number, 0-based
# connection type is either 'fixed' or 'pinned'
# node type is either 'support', 'joint', or 'internal'. 

x_unique = np.linspace(-B/2, B/2, Nf)

# create y coordinates: repeated -D/2 (Npw+1 times); from -D/2 to 0 @ plr; from 0 to D/2 @ plr; D/2 (Npw+1 times)
y_AB = np.repeat(-D/2, Npw+1)
y_BC = np.linspace(-D/2, 0, N_panelPerRoofIncline+1)[1:]
y_CD = np.linspace(0, D/2, N_panelPerRoofIncline+1)[1:]
y_DE = np.repeat(D/2, Npw+1)[1:]
y_perFrame = np.concatenate([y_AB, y_BC, y_CD, y_DE])
print(f"y_AB: {y_AB.shape}, y_BC: {y_BC.shape}, y_CD: {y_CD.shape}, y_DE: {y_DE.shape}")
print(f"total no. of nodes = {len(y_perFrame)}")

# for z, the height of the frame is divided into Npw panels for walls and Npr panels for roofs (the rise of the roof in z direction is D/2*tan(roofSlope_rad))
z_AB = np.linspace(0, He, Npw+1)
z_BC = np.linspace(He, Hr, N_panelPerRoofIncline+1)[1:]
z_CD = np.linspace(Hr, He, N_panelPerRoofIncline+1)[1:]
z_DE = np.linspace(He, 0, Npw+1)[1:]
z_perFrame = np.concatenate([z_AB, z_BC, z_CD, z_DE])
print(f"z_AB: {z_AB.shape}, z_BC: {z_BC.shape}, z_CD: {z_CD.shape}, z_DE: {z_DE.shape}")
print(f"total no. of nodes = {len(z_perFrame)}")

connectionType_at_ABCDE = ['fixed', 'pinned', 'fixed', 'pinned', 'fixed'] # the rest are always 'fixed'
nodeType_at_ABCDE = ['support', 'joint', 'joint', 'joint', 'support'] # the rest are always 'internal'

# mjIdxs = [0, Npw, Npw+N_panelPerRoofIncline-1, Npw+2*N_panelPerRoofIncline-2, Npw+2*N_panelPerRoofIncline+Npw-2]
mjIdxs = [0, Npw, Npw+N_panelPerRoofIncline, Npw+2*N_panelPerRoofIncline, Npw+2*N_panelPerRoofIncline+Npw]

# interlace connectionType_at_ABCDE with 'fixed' for the rest of the nodes
connectionType_perFrame = np.full(len(y_perFrame), 'fixed', dtype=object)
connectionType_perFrame[mjIdxs] = connectionType_at_ABCDE

# interlace nodeType_at_ABCDE with 'internal' for the rest of the nodes
nodeType_perFrame = np.full(len(y_perFrame), 'internal', dtype=object)
nodeType_perFrame[mjIdxs] = nodeType_at_ABCDE

# create a list of empty strings for the node names and replace the main nodes with A, B, C, D, & E
node_names = np.full(len(y_perFrame), '', dtype=object)
node_names[mjIdxs] = ['A', 'B', 'C', 'D', 'E']

# create a matrix of dof fixedness for each node (each has 6 dofs)
dof_fixed = np.zeros((len(y_perFrame), 6), dtype=int)
def connectionType_to_dof_fixedness(connectionType):
    if connectionType == 'fixed':
        return np.array([1,1,1,1,1,1])
    else:
        return np.array([0,0,0,1,1,1])
dof_fixed = np.array([connectionType_to_dof_fixedness(connectionType) for connectionType in connectionType_perFrame])

df_nodes = pd.DataFrame()
# columns: ID, name, frameID, connectionType, nodeType, x, y, z, dof_0, dof_1, dof_2, dof_3, dof_4, dof_5
nodeID = 0 # 0-based, continuous across all frames
for i, x in enumerate(x_unique):
    df_temp = pd.DataFrame()
    df_temp['ID'] = np.arange(nodeID, nodeID+len(y_perFrame))
    df_temp['name'] = [f"{node_names[mjIdx]}{i}" if mjIdx in mjIdxs else '-' for mjIdx in range(len(node_names))]
    df_temp['frameID'] = i
    # make sure the string is not cut off
    df_temp['connectionType'] = connectionType_perFrame
    df_temp['nodeType'] = nodeType_perFrame
    df_temp['x'] = y_perFrame
    df_temp['y'] = x
    df_temp['z'] = z_perFrame
    df_temp['dof_0'] = dof_fixed[:,0]
    df_temp['dof_1'] = dof_fixed[:,1]
    df_temp['dof_2'] = dof_fixed[:,2]
    df_temp['dof_3'] = dof_fixed[:,3]
    df_temp['dof_4'] = dof_fixed[:,4]
    df_temp['dof_5'] = dof_fixed[:,5]
    df_nodes = pd.concat([df_nodes, df_temp], axis=0)
    nodeID += len(y_perFrame)
df_nodes.reset_index(drop=True, inplace=True)
df_nodes['ID'] = df_nodes['ID'].astype(int)
df_nodes['frameID'] = df_nodes['frameID'].astype(int)
df_nodes['x'] = df_nodes['x'].astype(float)
df_nodes['y'] = df_nodes['y'].astype(float)
df_nodes['z'] = df_nodes['z'].astype(float)
print('')
print(df_nodes.head(35))

# write the dataframe to a csv file
df_nodes.to_csv(exportFolder + "nodes.csv", index=False)

y_AB: (6,), y_BC: (6,), y_CD: (6,), y_DE: (5,)
total no. of nodes = 23
z_AB: (6,), z_BC: (6,), z_CD: (6,), z_DE: (5,)
total no. of nodes = 23

    ID name  frameID connectionType  nodeType        x       y         z  \
0    0   A0        0          fixed   support -0.09150 -0.1372  0.000000   
1    1    -        0          fixed  internal -0.09150 -0.1372  0.015619   
2    2    -        0          fixed  internal -0.09150 -0.1372  0.031238   
3    3    -        0          fixed  internal -0.09150 -0.1372  0.046856   
4    4    -        0          fixed  internal -0.09150 -0.1372  0.062475   
5    5   B0        0         pinned     joint -0.09150 -0.1372  0.078094   
6    6    -        0          fixed  internal -0.07625 -0.1372  0.078412   
7    7    -        0          fixed  internal -0.06100 -0.1372  0.078729   
8    8    -        0          fixed  internal -0.04575 -0.1372  0.079047   
9    9    -        0          fixed  internal -0.03050 -0.1372  0.079365   
10  10    -        0 

In [27]:
'''
------------------------------------------------------------------------------------------
===================================== ELEMENTS ===========================================
------------------------------------------------------------------------------------------
create a dataframe for the elements
an element is a member connecting two main nodes of a frame and includes the internal nodes in between
defined by: ID, name, frameID, startNode, endNode, internalNodes
ID is 0-based and continuous across all frames
name is the name of the element composed of the names of the start and end nodes without spaces or hyphens
Nodes are identified by their ID
A single portal frame has four elements: AiBi, BiCi, CiDi, DiEi, where i is the frame number
'''


A_idx = 0
B_idx = Npw
C_idx = Npw + N_panelPerRoofIncline
D_idx = Npw + 2*N_panelPerRoofIncline
E_idx = Npw + 2*N_panelPerRoofIncline + Npw

# for the four elements, 
# the start nodes are:  A, B, C, D multiplied by the number of frames
# the end nodes are:    B, C, D, E multiplied by the number of frames
# the name is a combination of the start and end node names

df_elem = pd.DataFrame()
# columns: ID, name, frameID, startNode, endNode, internalNodes
elemID = 0
for i, x in enumerate(x_unique):
    sFrmIdx = i*len(y_perFrame)
    eFrmIdx = (i+1)*len(y_perFrame)
    iA = sFrmIdx + A_idx
    iB = sFrmIdx + B_idx
    iC = sFrmIdx + C_idx
    iD = sFrmIdx + D_idx
    iE = sFrmIdx + E_idx
    
    df_temp = pd.DataFrame()
    # add the next four consecutive numbers for the IDs
    df_temp['ID'] = np.arange(elemID, elemID+N_elem_perFrame)
    # combine the names of the start and end nodes to get the name of the element
    df_temp['name'] = [f"{df_nodes['name'].values[iA]}-{df_nodes['name'].values[iB]}", 
                       f"{df_nodes['name'].values[iB]}-{df_nodes['name'].values[iC]}", 
                       f"{df_nodes['name'].values[iC]}-{df_nodes['name'].values[iD]}", 
                       f"{df_nodes['name'].values[iD]}-{df_nodes['name'].values[iE]}"]
    
    df_temp['frameID'] = i
    # get the node IDs at A, B, C, & D for the four elements of this frame
    df_temp['startNode'] = df_nodes[df_nodes['frameID']==i]['ID'].values[[A_idx, B_idx, C_idx, D_idx]]
    df_temp['endNode'] = df_nodes[df_nodes['frameID']==i]['ID'].values[[B_idx, C_idx, D_idx, E_idx]]
    # get the node IDs for the internal nodes between the start and end nodes, nodes between iA and iB (excluding them), iB and iC (excluding them), iC and iD (excluding them), and iD and iE (excluding them)
    df_temp['internalNodes'] = [df_nodes[(df_nodes['frameID']==i) & (df_nodes['ID'] > iA) & (df_nodes['ID'] < iB)]['ID'].values,
                                 df_nodes[(df_nodes['frameID']==i) & (df_nodes['ID'] > iB) & (df_nodes['ID'] < iC)]['ID'].values,
                                 df_nodes[(df_nodes['frameID']==i) & (df_nodes['ID'] > iC) & (df_nodes['ID'] < iD)]['ID'].values,
                                 df_nodes[(df_nodes['frameID']==i) & (df_nodes['ID'] > iD) & (df_nodes['ID'] < iE)]['ID'].values]
    df_elem = pd.concat([df_elem, df_temp], axis=0)
    elemID += len(df_temp)
df_elem.reset_index(drop=True, inplace=True)
df_elem['ID'] = df_elem['ID'].astype(int)
df_elem['frameID'] = df_elem['frameID'].astype(int)
print('')
print(df_elem)

# write the dataframe to a csv file
df_elem.to_csv(exportFolder + "elements.csv", index=False)



    ID   name  frameID  startNode  endNode              internalNodes
0    0  A0-B0        0          0        5               [1, 2, 3, 4]
1    1  B0-C0        0          5       11           [6, 7, 8, 9, 10]
2    2  C0-D0        0         11       17       [12, 13, 14, 15, 16]
3    3  D0-E0        0         17       22           [18, 19, 20, 21]
4    4  A1-B1        1         23       28           [24, 25, 26, 27]
5    5  B1-C1        1         28       34       [29, 30, 31, 32, 33]
6    6  C1-D1        1         34       40       [35, 36, 37, 38, 39]
7    7  D1-E1        1         40       45           [41, 42, 43, 44]
8    8  A2-B2        2         46       51           [47, 48, 49, 50]
9    9  B2-C2        2         51       57       [52, 53, 54, 55, 56]
10  10  C2-D2        2         57       63       [58, 59, 60, 61, 62]
11  11  D2-E2        2         63       68           [64, 65, 66, 67]
12  12  A3-B3        3         69       74           [70, 71, 72, 73]
13  13  B3-C3      

In [28]:
'''
------------------------------------------------------------------------------------------
===================================== PANELS =============================================
------------------------------------------------------------------------------------------
create a dataframe for the panels
A panel is a surface defined by four nodes
defined by: ID, face, nodes

Nodes are sorted in a clockwise direction when looking at the panel from the outside starting with the bottom-left node

A frame goes from Face 4 (wall), to Face 2 (roof), to Face 1 (roof), and finally to Face 6 (wall)
Panels belong to faces as follows:
    Between A and B: Face 4
    Between B and C: Face 2
    Between C and D: Face 1
    Between D and E: Face 6

Example:
    Suppose the number of nodes per frame is Nnf. The nodes of ith frame are numbered from i*Nnf to (i+1)*Nnf-1. The number of panels per a pair of consecutive frames is Npf = Nnf-1. The panels of ith frame as its left edge are numbered from i*Npf to (i+1)*Npf-1. 
    
    A j-th panel between frames i and i+1 is formed by the four nodes with IDs: i*Nnf + j, i*Nnf + j+1, (i+1)*Nnf + j+1, and (i+1)*Nnf + j. 
'''

Nppf = 2*(Npw + N_panelPerRoofIncline) # number of panels per a pair of frames
print(f"number of panels per wall = {Npw}, \nnumber of panels per roof incline = {N_panelPerRoofIncline}, \nnumber of panels per frame = {Nppf}")

df_panels = pd.DataFrame()
# columns: ID, face, nodes

panelID = 0
for i, x in enumerate(x_unique[:-1]):
    sFrmIdx = i*len(y_perFrame)
    eFrmIdx = (i+1)*len(y_perFrame)
    print(f"frame {i}: {sFrmIdx} to {eFrmIdx}")
    for j in range(Nppf):
        i0 = sFrmIdx + j
        i1 = sFrmIdx + j + 1
        i2 = eFrmIdx + j + 1
        i3 = eFrmIdx + j
        df_temp = pd.DataFrame()
        df_temp['ID'] = [panelID]
        # concatenate of [faceIDs[0] Npw times, faceIDs[1] N_panelPerRoofIncline times, faceIDs[2] N_panelPerRoofIncline times, faceIDs[3] Npw times]
        df_temp['face'] = [np.concatenate([np.full(Npw, face_IDs[0]), np.full(N_panelPerRoofIncline, face_IDs[1]), np.full(N_panelPerRoofIncline, face_IDs[2]), np.full(Npw, face_IDs[3])])[j]]
        # df_temp['nodes'] = [[df_nodes['ID'].values[i0], df_nodes['ID'].values[i1], df_nodes['ID'].values[i2], df_nodes['ID'].values[i3]]]
        df_temp['nodes'] = [[i0, i1, i2, i3]]
        df_panels = pd.concat([df_panels, df_temp], axis=0)
        panelID += 1
df_panels.reset_index(drop=True, inplace=True)
df_panels['ID'] = df_panels['ID'].astype(int)
print('')
print(df_panels.head(40))

# write the dataframe to a csv file
df_panels.to_csv(exportFolder + "panels.csv", index=False)



number of panels per wall = 5, 
number of panels per roof incline = 6, 
number of panels per frame = 22
frame 0: 0 to 23
frame 1: 23 to 46
frame 2: 46 to 69
frame 3: 69 to 92

    ID  face             nodes
0    0     6    [0, 1, 24, 23]
1    1     6    [1, 2, 25, 24]
2    2     6    [2, 3, 26, 25]
3    3     6    [3, 4, 27, 26]
4    4     6    [4, 5, 28, 27]
5    5     1    [5, 6, 29, 28]
6    6     1    [6, 7, 30, 29]
7    7     1    [7, 8, 31, 30]
8    8     1    [8, 9, 32, 31]
9    9     1   [9, 10, 33, 32]
10  10     1  [10, 11, 34, 33]
11  11     2  [11, 12, 35, 34]
12  12     2  [12, 13, 36, 35]
13  13     2  [13, 14, 37, 36]
14  14     2  [14, 15, 38, 37]
15  15     2  [15, 16, 39, 38]
16  16     2  [16, 17, 40, 39]
17  17     4  [17, 18, 41, 40]
18  18     4  [18, 19, 42, 41]
19  19     4  [19, 20, 43, 42]
20  20     4  [20, 21, 44, 43]
21  21     4  [21, 22, 45, 44]
22  22     6  [23, 24, 47, 46]
23  23     6  [24, 25, 48, 47]
24  24     6  [25, 26, 49, 48]
25  25     6  [26,

In [29]:
'''
------------------------------------------------------------------------------------------
===================================== FRAMES =============================================
------------------------------------------------------------------------------------------
create a dataframe for the frames
A frame is a collection of elements and nodes
defined by: ID, name, nodes, elements

'''

df_frames = pd.DataFrame()
# columns: ID, name, nodes, elements
df_frames['ID'] = np.arange(Nf)
df_frames['name'] = [f"Frame-{i}" for i in range(Nf)]
df_frames['nodes'] = [df_nodes[df_nodes['frameID']==i]['ID'].values for i in range(Nf)]
df_frames['elements'] = [df_elem[df_elem['frameID']==i]['ID'].values for i in range(Nf)]
print('')
print(df_frames)

# write the dataframe to a csv file
df_frames.to_csv(exportFolder + "frames.csv", index=False)




   ID     name                                              nodes  \
0   0  Frame-0  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
1   1  Frame-1  [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3...   
2   2  Frame-2  [46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5...   
3   3  Frame-3  [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 8...   
4   4  Frame-4  [92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102...   

           elements  
0      [0, 1, 2, 3]  
1      [4, 5, 6, 7]  
2    [8, 9, 10, 11]  
3  [12, 13, 14, 15]  
4  [16, 17, 18, 19]  


In [30]:
# write all dataframes to a single excel file. Each dataframe will be written to a separate sheet
# add a 'summary' sheet that contains the building dimensions, number of nodes, elements, panels, and frames, etc.

summary = {
    'B': B,
    'D': D,
    'Hr': Hr,
    'roofSlope': roofSlope,
    'He': He,
    'Havg': Havg,
    'H': H,
    'g': g,
    'lScl': lScl,
    'Number of nodes': len(df_nodes),
    'Number of elements': len(df_elem),
    'Number of panels': len(df_panels),
    'Number of frames': len(df_frames),
    '': '',
    'Number of panels per wall': Npw,
    'Number of panels per roof incline': N_panelPerRoofIncline,
    'Number of elements per frame': N_elem_perFrame,
    'Number of panels per pair of frames': Nppf,
    'Number of nodes per frame': len(y_perFrame),
    'Number of panels per frame': Nppf,
    '': '',
    'Support nodes': 'A, E',
    'Joint nodes': 'B, C, D',
    'Connection type at ABCDE': 'fixed, pinned, fixed, pinned, fixed',
    'Node type at ABCDE': 'support, joint, joint, joint, support',
    '': '',
    'Building faces along the frame': str(face_IDs),   
}

with pd.ExcelWriter(exportFolder + "MWFRS_definition.xlsx") as writer:
    # write the summary sheet. List the contents of the summary dictionary over a pair of columns for the key-value pairs
    df_summary = pd.DataFrame(list(summary.items()), columns=['Parameter', 'Value'])
    df_summary.to_excel(writer, sheet_name='summary', index=False)
        
    df_nodes.to_excel(writer, sheet_name='nodes', index=False)
    df_elem.to_excel(writer, sheet_name='elements', index=False)
    df_panels.to_excel(writer, sheet_name='panels', index=False)
    df_frames.to_excel(writer, sheet_name='frames', index=False)

In [31]:
# read the excel file to check if the data was written correctly
summary_rd = pd.read_excel(exportFolder + "MWFRS_definition.xlsx", sheet_name='summary')

df_nodes_rd = pd.read_excel(exportFolder + "MWFRS_definition.xlsx", sheet_name='nodes')
df_elem_rd = pd.read_excel(exportFolder + "MWFRS_definition.xlsx", sheet_name='elements')
df_panels_rd = pd.read_excel(exportFolder + "MWFRS_definition.xlsx", sheet_name='panels')
df_frames_rd = pd.read_excel(exportFolder + "MWFRS_definition.xlsx", sheet_name='frames')

print('')
print(all(summary_rd == pd.DataFrame(list(summary.items()), columns=['Parameter', 'Value'])))

print('')
print(all(df_nodes_rd == df_nodes))
print('')
print(all(df_elem_rd == df_elem))
print('')
print(all(df_panels_rd == df_panels))
print('')
print(all(df_frames_rd == df_frames))



True

True

True

True

True


c:\Users\Tsinu\.conda\envs\wind\lib\site-packages\pandas\core\ops\array_ops.py:80: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.vec_compare(x.ravel(), y.ravel(), op)
